In [35]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd


cid ="0d262097ee284912b168027f224d160a" 
secret = "9d1d17eac2034913883c828fcddd83f8"


scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()
    
token = util.prompt_for_user_token(username,scope,client_id=cid,
                                   client_secret=secret, redirect_uri='http://localhost/')

sp = spotipy.Spotify(auth=token)

In [36]:
# defined a function that grabs all tracks of several playlists and combines into a single list

def get_playlist_tracks(playlists):
    t_tracks = []
    for playlist in playlists:
        t_playlist = sp.user_playlist_tracks(username, playlist_id=playlist, fields='items(track(id,name,uri))')['items']
        t_playlist = [x['track'] for x in t_playlist]
        t_tracks.extend(t_playlist)
    return t_tracks


In [37]:
# get tracks of several workout playlists motivation mix, beast mode, power workout, workout, hype, cardio, pumping iron, adrenaline workout, slay the weights
playlist_w = ['37i9dQZF1DXdxcBWuJkbcy', '37i9dQZF1DX76Wlfdnj7AP', '37i9dQZF1DWUVpAXiEPK8P', '37i9dQZF1DX70RN3TfWWJh', '37i9dQZF1DX4eRPd9frC1m','37i9dQZF1DWSJHnPb1f0X3','37i9dQZF1DWZYWNM3NfvzJ','37i9dQZF1DXe6bgV3TmZOL','37i9dQZF1DXbHrLkblwdd2']
workout_tracks = get_playlist_tracks(playlist_w)
workout_tracks = pd.DataFrame(workout_tracks) # turn dict into dataframe using pandas
workout_tracks = workout_tracks.drop_duplicates(subset = ['id']) # remove any duplicate songs
print(len(workout_tracks)) 


#get tracks of focus playlists Deep Focus, Brain Food, Intense Studying, Productive Morning, Music for Concentration, Perfect Concentration, Focus Flow
playlist_f = ['37i9dQZF1DWZeKCadgRdKQ', '37i9dQZF1DWXLeA8Omikj7', '37i9dQZF1DX8NTLI2TtZa6', '37i9dQZF1DX6T5dWVv97mp', '37i9dQZF1DX3PFzdbtx1Us', '37i9dQZF1DWSluGMsH1R9r','37i9dQZF1DWZZbwlv3Vmtr']
focus_tracks = get_playlist_tracks(playlist_f)
focus_tracks = pd.DataFrame(focus_tracks) 
focus_tracks = focus_tracks.drop_duplicates(subset = ['id'])
print(len(focus_tracks))


#get tracks of party playlists Dance Party, Girls Night, Party Jams, Throwback James, Ultimate Party Classics, TGIF, Beach Party, Dorm Daze, Party Deluxe
playlist_p = ['37i9dQZF1DXaXB8fQg7xif', '37i9dQZF1DX0Uv9tZ47pWo', '0ZWYUjUMhaq0Rammqq3qu1', '37i9dQZF1DX8ky12eWIvcW', '37i9dQZF1DX0IlCGIUGBsA','37i9dQZF1DXcRXFNfZr7Tp','37i9dQZF1DX4Y4RhrZqHhr','37i9dQZF1DXdfjr8o5AMIT','37i9dQZF1DWVWiyE9VDkCO']
party_tracks = get_playlist_tracks(playlist_p)
party_tracks = pd.DataFrame(party_tracks) 
party_tracks = party_tracks.drop_duplicates(subset = ['id'])
print(len(party_tracks))


#get tracks of chill playlists Relax and Unwind, Your Favorite Coffee House, Evening Chill, Hanging Out and Relaxing, Chill Vibes, Chillout Lounge
playlist_c = ['37i9dQZF1DWU0ScTcjJBdj','37i9dQZF1DX6ziVCJnEm59','37i9dQZF1DWZ0OzPeadl0h', '37i9dQZF1DXci7j0DJQgGp', '37i9dQZF1DX889U0CL85jj','37i9dQZF1DWTvNyxOwkztu']
chill_tracks = get_playlist_tracks(playlist_c)
chill_tracks = pd.DataFrame(chill_tracks) 
chill_tracks = chill_tracks.drop_duplicates(subset = ['id'])
print(len(chill_tracks))

535
536
601
542


In [38]:
def add_audio_features(x_tracks):
    x_ids = x_tracks[:]['id']
    features = []
    for id in x_ids:
        audio_x = sp.audio_features(tracks=id)
        features.extend(audio_x)
    merged_data = pd.merge(x_tracks, pd.DataFrame(features), on='id').drop(columns=['uri_y', 'track_href', 'type', 'analysis_url'])
    return merged_data

In [39]:
workout_features = add_audio_features(workout_tracks)
focus_features = add_audio_features(focus_tracks)
party_features = add_audio_features(party_tracks)
chill_features = add_audio_features(chill_tracks)

ConnectionError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/audio-features/?ids=72qlZsoVPpyFxVjL6GUuYI (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000290F6A01630>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
# flag each song with its corresponding playlist 
workout_features['playlist'] = 'workout'
focus_features['playlist'] = 'focus'
party_features['playlist'] = 'party'
chill_features['playlist'] = 'chill'

In [ ]:
# combine into one dataset
playlist_genres = [workout_features, focus_features, party_features, chill_features]
spotify_songs = pd.concat(playlist_genres)

In [ ]:
# search for missing values
spotify_songs.isna().sum()

In [ ]:
print(len(spotify_songs))
# inspecting duplicates
spotify_songs.nunique()

In [ ]:
# look for outliers
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

contin_columns = spotify_songs[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'speechiness','tempo','valence']]
for i, column in enumerate(contin_columns):
    plt.figure(i)
    sns.boxplot(x=spotify_songs[column])
    

In [ ]:
#export dataframe as csv for analysis
spotify_songs.to_csv('final_spotify_data.csv', index=False)